In [4]:
from google.colab import drive
drive.mount('/content/gdrive')
!pip install optuna

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
     |████████████████████████████████| 266kB 8.7MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 81kB 7.8MB/s 
     |████████████████████████████████| 163kB 16.0MB/s 
     |████████████████████████████████| 51kB 7.5MB/s 
     |████████████████████████████████| 133kB 13.6MB/s 
     |████████████████████████████████| 112kB 18.1MB/s 
     |████████████████████████████████| 81kB 9.4MB/s 
  Created wheel for optuna: filename=optuna-2.3.0-cp36-none-any.whl size=359761 sha256=17bb4557dc56e9647ed60857e53c58db3395860a5251335f69db01eadcdecf48
  Stored in directory: /root/.cache/pip/wheels/fa/91/19/64b0ec6b964f89c0695a9dc6db6f851d0b54c5381a5c9cadfb
Successfully built optuna
  Created wheel for PrettyTable: filename=prettytable-0.7.2-cp36-none-any

In [5]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import optuna
from sklearn.externals import joblib


def get_mnist_loaders(train_batch_size, test_batch_size):
    """Get MNIST data loaders"""
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=train_batch_size, shuffle=True)
    
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=test_batch_size, shuffle=True)

    return train_loader, test_loader

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning:

sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.



In [6]:
class Net(nn.Module):
    def __init__(self, activation):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)
        self.activation = activation 

    def forward(self, x):
        x = self.activation(self.conv1(x))
        x = self.conv2(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.activation(self.fc1(x))
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [7]:
def train(log_interval, model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data.to('cuda:0'))
        loss = F.nll_loss(output, target.to('cuda:0'))
        loss.backward()
        optimizer.step()
        
        if batch_idx % log_interval == 0:
          print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))  


    
def test(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data.to('cuda:0'))
            test_loss += F.nll_loss(output, target.to('cuda:0'), reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.to('cuda:0').view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
    return accuracy
          



In [ ]:
def train_mnist(trial):

  cfg = { 'device' : "cuda" if torch.cuda.is_available() else "cpu",
          'train_batch_size' : trial.suggest_int('batch_size', 16, 32, 64),
            'test_batch_size' : 1000 ,
            'n_epochs' : trial.suggest_int('num_epochs', 30, 50, 70),
            'seed' : 0,
            'log_interval' : 100,
            'save_model' : False,
            'lr' : trial.suggest_loguniform('lr', 1e-4, 1e-3),          
            'momentum': trial.suggest_uniform('momentum', 0.4, 0.99),
            'optimizer': trial.suggest_categorical('optimizer',[optim.SGD, optim.RMSprop , optim.Adam]),
            'activation': F.relu}
  torch.manual_seed(cfg['seed'])
  train_loader, test_loader = get_mnist_loaders(cfg['train_batch_size'], cfg['test_batch_size'])
  model = Net(cfg['activation']).to('cuda:0')
  optimizer = cfg['optimizer'](model.parameters(), lr=cfg['lr'])
  
  for epoch in range(1, cfg['n_epochs'] + 1):
      train(cfg['log_interval'], model, train_loader, optimizer, epoch)
      test_accuracy = test(model, test_loader)
      # print(test_accuracy)
  if cfg['save_model']:
      torch.save(model.state_dict(), "mnist_cnn.pt")

  return test_accuracy

if __name__ == '__main__':

  sampler = optuna.samplers.TPESampler()
      
  study = optuna.create_study(sampler=sampler, direction='maximize')
  study.optimize(func=train_mnist, n_trials=20)
  joblib.dump(study, '/content/gdrive/My Drive/Colab_Data/studies/mnist_optuna.pkl')

[I 2020-12-29 22:06:00,326] A new study created in memory with name: no-name-c7a3464c-db71-4769-a1b4-b96fed842d67


/usr/local/lib/python3.6/dist-packages/optuna/distributions.py:569: UserWarning:

The distribution is specified by [16, 32] and step=64, but the range is not divisible by `step`. It will be replaced by [16, 16].

/usr/local/lib/python3.6/dist-packages/optuna/distributions.py:569: UserWarning:

The distribution is specified by [30, 50] and step=70, but the range is not divisible by `step`. It will be replaced by [30, 30].

/usr/local/lib/python3.6/dist-packages/optuna/distributions.py:430: UserWarning:

Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains <class 'torch.optim.sgd.SGD'> which is of type type.

/usr/local/lib/python3.6/dist-packages/optuna/distributions.py:430: UserWarning:

Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains <class 'torch.optim.rmsprop.RMSprop'> which is of type type.

/usr/local/lib/python3.6/dist-packages/o

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw


...


Test set: Average loss: 0.0384, Accuracy: 9914/10000 (99%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.268284
Train Epoch: 1 [1600/60000 (3%)]	Loss: 0.676486
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.407428
Train Epoch: 1 [4800/60000 (8%)]	Loss: 0.138319
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.325428
Train Epoch: 1 [8000/60000 (13%)]	Loss: 0.134394
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.072670
Train Epoch: 1 [11200/60000 (19%)]	Loss: 0.057073
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.131746
Train Epoch: 1 [14400/60000 (24%)]	Loss: 0.087143
Train Epoch: 1 [16000/60000 (27%)]	Loss: 0.116715
Train Epoch: 1 [17600/60000 (29%)]	Loss: 0.085059
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.479784
Train Epoch: 1 [20800/60000 (35%)]	Loss: 0.125422
Train Epoch: 1 [22400/60000 (37%)]	Loss: 0.464263
Train Epoch: 1 [24000/60000 (40%)]	Loss: 0.180600
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.283715
Train Epoch: 1 [27200/60000 (45%)]	Loss: 0.399756
Train Epoch: 1 [28800/60000 (48%)]	Loss: 0.048580
Tra

[I 2020-12-30 00:19:32,963] Trial 12 finished with value: 99.07 and parameters: {'batch_size': 16, 'num_epochs': 30, 'lr': 0.0001611280292351388, 'momentum': 0.5549400600321446, 'optimizer': <class 'torch.optim.adam.Adam'>}. Best is trial 10 with value: 99.16.



Test set: Average loss: 0.0434, Accuracy: 9907/10000 (99%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.268284
Train Epoch: 1 [1600/60000 (3%)]	Loss: 0.641086
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.380456
Train Epoch: 1 [4800/60000 (8%)]	Loss: 0.123948
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.370241
Train Epoch: 1 [8000/60000 (13%)]	Loss: 0.105880
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.073127
Train Epoch: 1 [11200/60000 (19%)]	Loss: 0.061828
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.145414
Train Epoch: 1 [14400/60000 (24%)]	Loss: 0.099214
Train Epoch: 1 [16000/60000 (27%)]	Loss: 0.128738
Train Epoch: 1 [17600/60000 (29%)]	Loss: 0.202623
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.374455
Train Epoch: 1 [20800/60000 (35%)]	Loss: 0.113243
Train Epoch: 1 [22400/60000 (37%)]	Loss: 0.355672
Train Epoch: 1 [24000/60000 (40%)]	Loss: 0.171795
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.299512
Train Epoch: 1 [27200/60000 (45%)]	Loss: 0.369127
Train Epoch: 1 [28800/60000 (48%)]	Loss: 0.054297
Tra

[I 2020-12-30 00:29:59,966] Trial 13 finished with value: 99.03 and parameters: {'batch_size': 16, 'num_epochs': 30, 'lr': 0.000170539011462402, 'momentum': 0.7084792797858638, 'optimizer': <class 'torch.optim.adam.Adam'>}. Best is trial 10 with value: 99.16.



Test set: Average loss: 0.0423, Accuracy: 9903/10000 (99%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.268284
Train Epoch: 1 [1600/60000 (3%)]	Loss: 0.853311
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.473771
Train Epoch: 1 [4800/60000 (8%)]	Loss: 0.179897
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.420567
Train Epoch: 1 [8000/60000 (13%)]	Loss: 0.148168
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.099263
Train Epoch: 1 [11200/60000 (19%)]	Loss: 0.091233
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.315166
Train Epoch: 1 [14400/60000 (24%)]	Loss: 0.107448
Train Epoch: 1 [16000/60000 (27%)]	Loss: 0.145346
Train Epoch: 1 [17600/60000 (29%)]	Loss: 0.082672
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.598575
Train Epoch: 1 [20800/60000 (35%)]	Loss: 0.175273
Train Epoch: 1 [22400/60000 (37%)]	Loss: 0.438136
Train Epoch: 1 [24000/60000 (40%)]	Loss: 0.229314
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.343572
Train Epoch: 1 [27200/60000 (45%)]	Loss: 0.435985
Train Epoch: 1 [28800/60000 (48%)]	Loss: 0.146233
Tra